In [ ]:
# Import python packages
from snowflake.core import Root

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()
env = 'dev'
session.use_schema('DEMO_DB.DATA')

In [ ]:
from snowflake.core.task.dagv1 import DAGOperation, DAG, DAGTask
from datetime import timedelta

# DAG API 를 사용해서 task 생성
warehouse_name = "DEMO_WH"
dag_name = "DEMO_DAG"

api_root = Root(session)
schema = api_root.databases['DEMO_DB'].schemas['DATA']
dag_op = DAGOperation(schema)

# Define the DAG
with DAG(dag_name, schedule=timedelta(days=1), warehouse=warehouse_name) as dag:
    dag_task1 = DAGTask("stats_customer", definition=f'''EXECUTE NOTEBOOK DEMO_DB.DATA."{env}_STATS_CUSTOMER"()''', warehouse=warehouse_name)
    dag_task2 = DAGTask("stats_sales", definition=f'''EXECUTE NOTEBOOK DEMO_DB.DATA."{env}_STATS_SALES"()''', warehouse=warehouse_name)

    # Define the dependencies between the tasks
    dag_task1 >> dag_task2 # dag_task1 is a predecessor of dag_task2

# Create the DAG in Snowflake
dag_op.deploy(dag, mode="orreplace")

In [ ]:
dagiter = dag_op.iter_dags(like='demo_dag%')
for dag_name in dagiter:
    print(dag_name)

dag_op.run(dag)